In [1]:
import os
os.chdir('/Users/khalilouardini/Desktop/projects/scVI/scvi')
from unittest import TestCase
from anndata import AnnData
import numpy as np
from dataset.tree import TreeDataset
from dataset.anndataset import AnnDatasetFromAnnData
from inference.tree_inference import TreeTrainer
from models.treevae import TreeVAE
import torch
import pdb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ete3 import Tree

tree_name = "data/Cassiopeia_trees/lg7_tree_hybrid_priors.alleleThresh.processed.txt"
with open (tree_name, "r") as myfile:
    tree_string = myfile.readlines()
    
tree = Tree(tree_string[0], 1)
leaves = tree.get_leaves()

print(tree)


      /- /- /-RE.CTAGAGTCAGTAAGAT-1
   /-|
  |  |   /- /- /-RE.TTAACTCTCCCAGGTG-1
  |   \-|
  |      \- /- /-RE.CGGTTAATCTATCCCG-1
  |
  |      /- /-M1.CTCAGAAGTCTCCATC-1
  |     |
  |   /-|      /- /- /- /-LL.GGTGCGTGTCTAGTGT-1
  |  |  |     |
  |  |   \- /-|-- /- /- /-LL.GAATAAGTCGTCACGG-1
  |  |        |
  |  |        |      /- /- /-M1.CGTGAGCTCCTGTACC-1
  |  |         \- /-|
  |  |               \- /- /-M1.TTCCCAGTCGTGGTCG-1
  |--|
  |  |      /- /- /- /- /-M1.ATTGGACAGATAGGAG-1
  |  |     |
  |  |     |      /- /- /- /- /- /-RE.GCTCCTACACTTCTGC-1
  |  |     |     |
  |  |     |     |   /- /- /-RE.TGACTTTAGCACGCCT-1
  |  |     |     |--|
  |  |     |     |  |   /- /- /-RE.CATCGAACATGGGAAC-1
  |   \- /-|   /-|   \-|
  |        |  |  |      \- /-RE.CGTCTACTCTCGTTTA-1
  |        |  |  |
  |        |  |  |         /- /- /- /- /-RE.TCATTACAGCTAGCCC-1
  |        |  |  |      /-|
  |        |  |  |     |   \- /-RE.AGCATACGTCTGCCAG-1
  |        |  |   \- /-|
  |        |  |        |   /- 

In [4]:
#gene_dataset = GeneExpressionDataset()
x = np.random.randint(1, 100, (len(leaves), 10))
adata = AnnData(x)
gene_dataset = AnnDatasetFromAnnData(adata)
#gene_dataset.populate_from_data(adata)
barcodes = [l.name for l in tree.get_leaves()]
gene_dataset.initialize_cell_attribute('barcodes', barcodes)

[2020-09-20 16:38:08,109] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-09-20 16:38:08,111] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-09-20 16:38:08,113] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-09-20 16:38:08,114] INFO - scvi.dataset.dataset | Downsampled from 663 to 663 cells


In [5]:
cas_dataset = TreeDataset(gene_dataset, tree_name=tree_name)

[2020-09-20 16:38:15,345] INFO - scvi.dataset.dataset | Merging datasets. Input objects are modified in place.
[2020-09-20 16:38:15,346] INFO - scvi.dataset.dataset | Gene names and cell measurement names are assumed to have a non-null intersection between datasets.
[2020-09-20 16:38:15,347] INFO - scvi.dataset.dataset | Keeping 10 genes
[2020-09-20 16:38:15,347] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-09-20 16:38:15,348] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-09-20 16:38:15,349] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-09-20 16:38:15,350] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-09-20 16:38:15,351] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-09-20 16:38:15,353] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-09-20 16:38:15,354] INFO - scvi.dataset.dataset | Downsampled from 663 to 663 cells


In [6]:
n_epochs = 50
lr = 1e-3
# No batches beacause of the message passing
use_batches = True
use_cuda = False

In [7]:
cas_dataset.n_batches * use_batches

1

In [8]:
vae = TreeVAE(cas_dataset.nb_genes,
              tree = cas_dataset.tree,
              n_batch=cas_dataset.n_batches * use_batches,
              n_latent=2,
              use_clades=False
             )

In [9]:
trainer = TreeTrainer(
    model = vae,
    gene_dataset = cas_dataset,
    train_size=0.8,
    use_cuda=use_cuda,
    frequency=5,
    batch_size=1
)

train_leaves:  [[1], [3], [4], [5], [6], [7], [10], [11], [13], [14], [15], [16], [17], [18], [19], [20], [21], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [43], [44], [45], [46], [47], [49], [50], [51], [55], [57], [58], [59], [60], [61], [62], [64], [65], [66], [68], [69], [70], [71], [73], [74], [75], [76], [77], [78], [79], [81], [82], [83], [84], [85], [86], [87], [88], [90], [91], [92], [93], [95], [96], [97], [98], [100], [101], [102], [103], [104], [105], [106], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [120], [121], [122], [123], [125], [126], [127], [128], [129], [130], [131], [132], [133], [134], [135], [137], [138], [139], [141], [143], [144], [145], [146], [147], [148], [149], [151], [152], [153], [154], [155], [156], [157], [158], [159], [160], [161], [162], [163], [164], [165], [167], [169], [170], [172], [173], [175], [176], [177], [178], [179], [180], [182], [183], [184], [185], [186], [187], [18

In [10]:
posterior = trainer.create_posterior(trainer.model, cas_dataset, trainer.clades)

In [11]:
import copy
import torch

data = [x for x in posterior.data_loader]
x = data[0]

batch_index = torch.from_numpy(np.array([[0]]*67))
y = copy.deepcopy(batch_index)

outputs = vae.inference(x[0], batch_index, y)
z = outputs["z"]

In [15]:
z.shape

torch.Size([663, 2])

In [19]:
# message passing likelihood
vae.initialize_visit()
#torch.cat((torch.FloatTensor([0] * (N - len(leaves)) * d).reshape(-1, d), z), 0)
vae.initialize_messages(
    z,
    cas_dataset.barcodes,
    z.shape[1]
)

pdb.set_trace()

--Return--
> <ipython-input-19-9db0a6946d86>(10)<module>()->None
-> pdb.set_trace()
(Pdb) vae.tree
Tree node 'prior_root' (0x1a31d576d)
(Pdb) vae.root
'0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0'
(Pdb) vae.root.name
*** AttributeError: 'str' object has no attribute 'name'
(Pdb) x = [n for n in vae.tree.traverse("levelorder")]
(Pdb) x[0].name
'prior_root'
(Pdb) x[1].name
'0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0'
(Pdb) x[1]["mu"]
*** TypeError: 'TreeNode' object is not subscriptable
(Pdb) x[1].mu
tensor([0., 0.], dtype=torch.float64)
(Pdb) [x[k].mu for y in k in len(x)]
*** NameError: name 'k' is not defined
(Pdb) [x[k].mu for k in len(x)]
*** TypeError: 'int' object is not iterable
(Pdb) [x[k].mu for k in range(len(x))]
[tensor([0., 0.], dtype=torch.float64), tensor([0., 0.], dtype=torch.float64), tensor([0., 0.], dtype=torch.float64), tensor([0., 0.], dtype=torch.float64), tenso

(Pdb) q


BdbQuit: 

In [ ]:
vae.perform_message_passing((vae.tree & vae.root), z.shape[1], False)

mp_lik = vae.aggregate_messages_into_leaves_likelihood(
            z.shape[1],
            add_prior=False
        )

print("Message passing output O(nd): ", 
      mp_lik) 

In [13]:
print(vae.tree)


         /- /- /-RE.CTAGAGTCAGTAAGAT-1
      /-|
     |  |   /- /- /-RE.TTAACTCTCCCAGGTG-1
     |   \-|
     |      \- /- /-RE.CGGTTAATCTATCCCG-1
     |
     |      /- /-M1.CTCAGAAGTCTCCATC-1
     |     |
     |   /-|      /- /- /- /-LL.GGTGCGTGTCTAGTGT-1
     |  |  |     |
     |  |   \- /-|-- /- /- /-LL.GAATAAGTCGTCACGG-1
     |  |        |
     |  |        |      /- /- /-M1.CGTGAGCTCCTGTACC-1
     |  |         \- /-|
     |  |               \- /- /-M1.TTCCCAGTCGTGGTCG-1
     |--|
     |  |      /- /- /- /- /-M1.ATTGGACAGATAGGAG-1
     |  |     |
     |  |     |      /- /- /- /- /- /-RE.GCTCCTACACTTCTGC-1
     |  |     |     |
     |  |     |     |   /- /- /-RE.TGACTTTAGCACGCCT-1
     |  |     |     |--|
     |  |     |     |  |   /- /- /-RE.CATCGAACATGGGAAC-1
     |   \- /-|   /-|   \-|
     |        |  |  |      \- /-RE.CGTCTACTCTCGTTTA-1
     |        |  |  |
     |        |  |  |         /- /- /- /- /-RE.TCATTACAGCTAGCCC-1
     |        |  |  |      /-|
     |        |  |  |    

In [16]:
len(trainer.clades)

663

In [22]:
for n in vae.tree.traverse("levelorder"):
    if n.is_leaf():
        print("leaf", n.mu, n.name)
    else:
        print("not leaf", n.mu, n.name)

not leaf tensor([0., 0.], dtype=torch.float64) prior_root
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|3|0|0|0|-|-|-|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 3|4|0|5|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|2|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|2|0|0|0|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|2|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|2|0|0|0|0|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) -|-|-|-|-|-|0|0|0|0|0|0|4|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|-|-|-|0|0|0|-|-|-|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|3|4|0|0|0|0|0|0

not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|2|0|0|3|0|0|0|0|0|0|0|0|0|0|0|-|-|-|3|0|2|3|0|2|0|0|0|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|2|0|0|3|0|0|0|0|0|0|0|0|0|0|0|-|-|-|3|0|0|0|0|2|5|0|0|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|2|-|-|-|0|0|0|0|0|0|-|-|-|0|0|0|0|0|3|0|0|0|0|2|5|0|0|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|2|4|0|3|0|0|0|0|0|0|0|0|0|0|0|-|-|-|3|0|0|0|0|2|0|0|0|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|2|4|0|3|0|0|0|0|0|0|0|0|0|0|0|0|0|0|3|0|0|0|0|2|-|-|-|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|2|4|0|3|0|0|0|0|0|0|0|0|0|0|0|0|0|0|-|-|-|0|0|2|0|0|0|-|-|-|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|0|0|0|0|0|4|0|0|0|2|4|0|3|0|0|0|0|0|0|0|0|0|0|0|0|0|0|-|-|-|0|0|2|0|0|0|2|0|0|0|0

not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|4|0|0|2|3|0|0|0|2|2|0|-|-|-|-|-|0|0|0|0|2|2|2|0|0|-|-|0|0|0|0|0|0|0|0|2|-|-|-|4|8|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|4|0|0|2|3|-|-|-|-|-|-|0|2|0|3|0|0|0|0|0|2|2|2|0|0|0|0|0|0|0|0|0|0|0|0|2|-|-|-|4|8|0|0|0
leaf tensor([-0.5285, -0.3980], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.AGTAGTCTCACCTCGT-1
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|4|0|0|2|3|0|0|0|2|2|0|0|2|0|0|0|0|0|0|0|2|2|2|0|0|-|-|0|0|0|0|0|0|0|0|2|-|-|-|-|-|-|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|4|0|0|2|3|0|0|0|-|-|-|0|2|0|0|0|-|-|-|0|2|2|2|0|0|0|0|0|0|0|0|0|0|0|0|2|-|-|-|-|-|-|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|2|0|0|0|0|-|-|-|0|2|0|0|0|0|0|0|0|2|2|2|0|0|-|-|0|0|0|0|0|0|0|0|2|-|-|-|-|-|-|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|2|0|0|0|0|8|5|0|0|2|0|0|0|0|0|0|-|-|-|2|0|0|0|0|0|0|0|-|-|-|0|0|2|-|-|-|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.floa

not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|4|0|0|2|0|0|0|0|2|2|0|-|-|0|0|0|0|0|0|0|2|0|2|0|0|-|-|0|0|0|0|0|0|0|0|2|-|-|-|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|4|0|0|2|0|0|0|0|2|2|0|0|2|0|0|0|0|0|0|0|2|0|-|-|-|-|-|0|0|0|0|0|0|0|0|2|-|-|-|2|0|0|0|0
leaf tensor([1.0229, 1.6787], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.GCTCCTACACTTCTGC-1
not leaf tensor([0., 0.], dtype=torch.float64) 3|4|0|5|0|0|0|0|0|-|-|-|0|2|0|0|0|0|8|0|2|0|0|3|5|0|0|0|0|-|-|6|0|0|-|-|-|0|3|0|-|-|-|-|-|-|0|0
leaf tensor([1.2178, 0.0631], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.AAATGCCTCGCTTAGA-1
not leaf tensor([0., 0.], dtype=torch.float64) 3|4|6|5|0|0|0|6|0|-|-|-|0|2|12|0|0|-|-|-|2|0|0|3|5|0|0|0|0|3|0|0|0|0|0|0|0|0|3|0|-|-|-|0|0|0|0|2
not leaf tensor([0., 0.], dtype=torch.float64) 3|4|6|5|0|0|0|6|0|-|-|-|0|2|0|0|0|0|0|0|2|0|0|3|5|0|0|0|0|3|0|-|-|-|0|0|0|0|3|0|-|-|-|3|0|0|0|2
not leaf tensor([0., 0.], dtype=torch.float64) 3|4|6|5|0|0|0|6|0|-|-|-|0|2|0|0|0|0|0|0|2

leaf tensor([-1.0903,  0.7214], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.AGCGGTCAGTACATGA-1
leaf tensor([ 2.0844, -0.8468], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.CAGTCCTGTCCGACGT-1
leaf tensor([-0.6668,  0.6278], dtype=torch.float64, grad_fn=<CopyBackwards>) M2.TGCGGGTCAAGTTGTC-1
leaf tensor([0.4247, 0.1985], dtype=torch.float64, grad_fn=<CopyBackwards>) M1.TCAGCTCTCAACACAC-1
leaf tensor([ 0.2051, -1.0641], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.ACTATCTAGGGATACC-1
leaf tensor([-0.1553, -0.3986], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.TACGGATAGCCACCTG-1
leaf tensor([-0.0732,  1.2792], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.CAGTAACCATGGGAAC-1
leaf tensor([-0.5667,  0.4331], dtype=torch.float64, grad_fn=<CopyBackwards>) RE.GAGGTGAGTCGAGTTT-1
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|0|2|2|2|0|0|2|0|0|0|2|0|0|0|0|0|0|0|2|0|2|9|0|-|-|0|0|4|-|-|-|3|0|2|-|-|-|2|0|0|0|0
not leaf tensor([0., 0.], dtype=torch.float64) 0|0|0|0|0|0|